# **Evaluation**

In [1]:
import Discount

In [2]:
DATA_PATH = '/Users/sascha/Desktop/mlim-g2/data/'
EXPORT_PATH = '/Users/sascha/Desktop/mlim-g2/export/'

# -------- ToDo --------
# use relative Paths? Would make sense if we include the generated parquets in the repository
# load function does not work with relative paths yet, because it can't handle dots (only in the filename)
# DATA_PATH = '/../data/'
# EXPORT_PATH = '/../export/'

In [3]:
files = {
    'data': [
        DATA_PATH + 'baskets.parquet',
        DATA_PATH + 'coupon_index.parquet', 
        DATA_PATH + 'coupons.parquet', 
        DATA_PATH + 'merged.parquet',
        DATA_PATH + 'clean.parquet' 
        #DATA_PATH + 'prepare.parquet'
    ],
    'mappings': [
        EXPORT_PATH + 'product_histories.parquet',
        EXPORT_PATH + 'prices.parquet'
    ]
}

In [4]:
discount_model = Discount.No_Cross_Effects()
discount_model.load(files)

discount_model.pipeline(90, 4, [0.15, 0.2, 0.25, 0.3], shoppers=(0,1999))
discount_model.top_coupons

KeyError: 'prepare'

In [4]:
discount_model = Discount.No_Cross_Effects()
discount_model.load(files)

discount_model.pipeline(90, 4, [0.15, 0.2, 0.25, 0.3], shoppers=(0,1999))
discount_model.top_coupons

[discount] train-log-loss: 0.076118548022094


,shopper,week,coupon,product,discount
1171,0,90,0,171,0.15
1076,0,90,1,76,0.15
1157,0,90,2,157,0.15
1067,0,90,3,67,0.15
1004,0,90,4,4,0.3
...,...,...,...,...,...
2499825,1999,90,0,75,0.15
2499963,1999,90,1,213,0.15
2499791,1999,90,2,41,0.15
2499803,1999,90,3,53,0.25


<br>

### **Evaluation class: first approach**

In [5]:
class Evaluation:
    
    def __init__(self, discount_model):
        self.model = discount_model
        self.model_input = {}
        
        
    def get_model_inputs(self):
        model = self.model
        top_coupons = model.top_coupons
    
        _, _, X_zero, _ = model.train_test_split(model.test_week, model.train_window, df=model.df, features=model.features)
        X_top_coupons = X_zero.copy()
        X_top_coupons.loc[top_coupons.index, 'discount'] = top_coupons['discount']
        X_top_coupons['discount'] = X_top_coupons['discount'].astype('float32')
        self.model_input['zero'] = X_zero
        self.model_input['top_coupons'] = X_top_coupons
        
        
    def get_total_expected_revenue(self, model_input='top_coupons'):
        import numpy as np
        
        if type(model_input) == str:
            model_input = self.model_input[model_input]
        model = self.model
        prices = model.data['prepare'].loc[model_input.index, 'price'].values
        probs = model.predict(model.model, model_input)
        return np.dot(probs, prices)
    

In [6]:
evaluation = Evaluation(discount_model)

In [7]:
evaluation.get_model_inputs()

In [8]:
evaluation.model_input['top_coupons']

,price,discount,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_WOE,product_WOE
1000,688.0,0.0,104.0,0.0,0.000000,0.0,0.000000,0.074996,-0.595444
1001,560.0,0.0,104.0,0.0,0.000000,0.0,0.000000,0.074996,0.615436
1002,773.0,0.0,104.0,0.0,0.000000,0.0,0.000000,0.074996,-0.713309
1003,722.0,0.0,104.0,0.0,0.000000,0.0,0.000000,0.074996,-1.397781
1004,620.0,0.3,20.0,0.0,0.000000,0.0,0.088889,0.074996,-0.163894
...,...,...,...,...,...,...,...,...,...
2499995,549.0,0.0,1.0,1.0,0.333333,0.2,0.133333,0.248261,-0.818372
2499996,702.0,0.0,104.0,0.0,0.000000,0.0,0.000000,0.248261,-0.305680
2499997,670.0,0.0,104.0,0.0,0.000000,0.0,0.000000,0.248261,-1.574931
2499998,490.0,0.0,104.0,0.0,0.000000,0.0,0.000000,0.248261,0.792432


In [9]:
evaluation.get_total_expected_revenue('zero')

8855312.150058173

In [10]:
evaluation.get_total_expected_revenue('top_coupons')

10560617.682029858